In [1]:
import pandas as pd

In [2]:
price=pd.read_excel(r"./A股1.xlsx",sheet_name="price")
price.head()

,stockcode,简称,tradedate,price
0,000001.SZ,平安银行,2000-01-28,18.50
1,000001.SZ,平安银行,2000-02-29,18.32
2,000001.SZ,平安银行,2000-03-31,18.37
3,000001.SZ,平安银行,2000-04-28,19.06
4,000001.SZ,平安银行,2000-05-31,18.00


In [3]:
pb=pd.read_excel(r"./A股1.xlsx",sheet_name='pb')
pb.head()

,stockcode,简称,tradedate,pb
0,000001.SZ,平安银行,2000-01-28,7.3283
1,000001.SZ,平安银行,2000-02-29,7.257
2,000001.SZ,平安银行,2000-03-31,7.2769
3,000001.SZ,平安银行,2000-04-28,10.1965
4,000001.SZ,平安银行,2000-05-31,9.6294


In [4]:
mkt=pd.read_excel(r"./A股1.xlsx",sheet_name='mkt')
mkt.head()


,stockcode,简称,tradedate,mkt
0,000001.SZ,平安银行,2000-01-28,28709171202
1,000001.SZ,平安银行,2000-02-29,28429838725
2,000001.SZ,平安银行,2000-03-31,28507431080
3,000001.SZ,平安银行,2000-04-28,29578205574
4,000001.SZ,平安银行,2000-05-31,27933247656


In [5]:
def panel_fill(data,time,individual,value):
    data=data.sort_values(by=[time,individual]).reset_index(drop=True)
    if data[value].isna().sum()==0:
        print("原始数据不含有缺失值")
        return data
    else:
        missing_sum=data[value].isna().sum()
        data_group=data.groupby(individual)
        data=data_group.transform(lambda x: x.interpolate(method='linear',limit_direction='both')).join(data.reset_index()[individual])
        if data[value].isna().sum()==0:
            print("原始数据缺失值数量为：",missing_sum,"，已处理")
            return data
        else:
            data=data[[time,individual]].join(data.groupby(time)[value].transform(lambda x: x.fillna(x.mean())).reset_index(drop=True))
            if data[value].isna().sum()==0:
                print("存在有组全部缺失，已处理")
                return data
            else:
                print("经处理后，仍含有缺失值")
                return data

In [6]:
pb['pb']=pd.to_numeric( pb['pb'],errors='coerce')
BM=pb.copy()
BM['BM']=1/BM['pb']
BM=panel_fill(BM,"tradedate",'stockcode','pb')

原始数据缺失值数量为： 129 ，已处理


In [7]:
ipodate = pd.read_excel(r"./上市日期.xlsx")
ipodate.head()

,stockcode,证券简称,ipodate
0,000005.SZ,ST星源,1990-12-10
1,600651.SH,飞乐音响,1990-12-19
2,600653.SH,申华控股,1990-12-19
3,600654.SH,ST中安,1990-12-19
4,600602.SH,云赛智联,1990-12-19


In [8]:
price['tradedate'] = pd.to_datetime(price.tradedate)
mkt['tradedate'] = pd.to_datetime(mkt.tradedate)
pb['tradedate'] = pd.to_datetime(pb.tradedate)
ipodate['ipodate'] = pd.to_datetime(ipodate.ipodate)

In [9]:
pb1 =pb.drop(pb[(pb['简称'].str.contains('ST') )|( pb['简称'].str.contains('ST*'))].index)
BM1 =BM.drop(BM[(BM['简称'].str.contains('ST') )|(BM['简称'].str.contains('ST*'))].index)
mkt1 =mkt.drop(mkt[(mkt['简称'].str.contains('ST') )|( mkt['简称'].str.contains('ST*'))].index)
price1 =price.drop(price[(price['简称'].str.contains('ST') )|( price['简称'].str.contains('ST*'))].index)

In [10]:
price1['tradedate'] = pd.to_datetime(price1.tradedate)
mkt1['tradedate'] = pd.to_datetime(mkt1.tradedate)
pb1['tradedate'] = pd.to_datetime(pb1.tradedate)
BM1['tradedate'] = pd.to_datetime(BM1.tradedate)

In [11]:
f=pd.merge(BM1,mkt1,on=['tradedate',"stockcode","简称"])
f=f.sort_values(by=['tradedate',"stockcode"]).reset_index(drop=True)
f.head()

,简称,tradedate,pb,BM,stockcode,mkt
0,平安银行,2000-01-28,7.3283,0.136457,000001.SZ,28709171202
1,万科A,2000-01-28,3.2113,0.311400,000002.SZ,6656753978
2,国华网安,2000-01-28,16.0083,0.062468,000004.SZ,1125287566
3,深振业A,2000-01-28,2.4339,0.410863,000006.SZ,2786972025
4,神州高铁,2000-01-28,68.7030,0.014555,000008.SZ,6702441928


In [12]:
import datetime
f['tradedate'] = pd.to_datetime(f.tradedate)
ipodate['ipodate'] = pd.to_datetime(ipodate.ipodate)

#先计算上市一年后的日期，然后用交易日和该日期进行比较，只保留上市一年后的数据
ipodate['entrydate']=ipodate['ipodate']+datetime.timedelta(365)

#将其合并进因子数据（左连接）
f=pd.merge(f,ipodate,how='left',left_on='stockcode',right_on='stockcode')

#选取数据
f=f.loc[f.tradedate>=f.entrydate].reset_index(drop=True)
del f['证券简称']
f.head()

,简称,tradedate,pb,BM,stockcode,mkt,ipodate,entrydate
0,平安银行,2000-01-28,7.3283,0.136457,000001.SZ,28709171202,1991-04-03,1992-04-02
1,万科A,2000-01-28,3.2113,0.311400,000002.SZ,6656753978,1991-01-29,1992-01-29
2,国华网安,2000-01-28,16.0083,0.062468,000004.SZ,1125287566,1991-01-14,1992-01-14
3,深振业A,2000-01-28,2.4339,0.410863,000006.SZ,2786972025,1992-04-27,1993-04-27
4,神州高铁,2000-01-28,68.7030,0.014555,000008.SZ,6702441928,1992-05-07,1993-05-07


In [13]:
f.drop(['ipodate','entrydate'],axis=1,inplace=True)
f.head()

,简称,tradedate,pb,BM,stockcode,mkt
0,平安银行,2000-01-28,7.3283,0.136457,000001.SZ,28709171202
1,万科A,2000-01-28,3.2113,0.311400,000002.SZ,6656753978
2,国华网安,2000-01-28,16.0083,0.062468,000004.SZ,1125287566
3,深振业A,2000-01-28,2.4339,0.410863,000006.SZ,2786972025
4,神州高铁,2000-01-28,68.7030,0.014555,000008.SZ,6702441928


In [14]:
f['ym']=f.tradedate.apply(lambda x : x.year*100 + x.month)
f.head()

,简称,tradedate,pb,BM,stockcode,mkt,ym
0,平安银行,2000-01-28,7.3283,0.136457,000001.SZ,28709171202,200001
1,万科A,2000-01-28,3.2113,0.311400,000002.SZ,6656753978,200001
2,国华网安,2000-01-28,16.0083,0.062468,000004.SZ,1125287566,200001
3,深振业A,2000-01-28,2.4339,0.410863,000006.SZ,2786972025,200001
4,神州高铁,2000-01-28,68.7030,0.014555,000008.SZ,6702441928,200001
